In [7]:
import openpyxl
import pprint
import re
import pandas as pd

#Excel読み込み
wb1 = openpyxl.load_workbook("moto_sample.xlsx")
wb2 = openpyxl.load_workbook("seikyu.xlsx")

#シートネーム全部リスト化
wb1sheet = [sheetname for sheetname in wb1.sheetnames]#請求元サンプル入ってる。
wb2sheet = [sheetname for sheetname in wb2.sheetnames]#請求書一覧データ

sheet1 = wb2["請求一覧2101"]
#3行目をカラムとしてリスト化
col = []
for row in sheet1.iter_cols():
    for cell in row:
        if cell.row == 3:
            col.append(cell.value)
print(col)

#各シートデータ、行最大値リスト化
sheet_list = []
max_row_list = []
for i,sheet in enumerate(wb2sheet):
    if re.search('請求一覧', sheet):
        sheet_row = wb2[wb2sheet[i]]
        sheet_list.append(sheet_row)
        max_row_list.append(sheet_row.max_row)

#列名置き換え用辞書型カラムデータ作成
dic_col = {}
for i,col in enumerate(col):
    dic_col[i] = col
 
print(dic_col)
print(sheet_list)
print(max_row_list)

#行情報取得関数（取得した行をリストに入れる）
def get_value_list(t_2d):
    return([[cell.value for cell in row] for row in t_2d])

def get_list_2d(sheet, start_row, end_row, start_col, end_col):
    return get_value_list(sheet.iter_rows(min_row=start_row,
                                          max_row=end_row,
                                          min_col=start_col,
                                          max_col=end_col))

#顧客管理テーブルシートを読み込んでデータフレーム化
cs = wb2["顧客管理テーブル"]
cs_list = get_list_2d(cs,3,cs.max_row,1,2)

cs_df = pd.DataFrame(cs_list)
cs_df = cs_df.dropna(how='all')
#請求一覧と合体させたいので顧客ＣＤを相手先コードにする
cs_df.columns = ['相手先コード','顧客名']
display(cs_df)


['受注No.', '相手先名', '相手先コード', '商品名１', '商品名２', '検収日', '金額（税抜き）', '備考']
{0: '受注No.', 1: '相手先名', 2: '相手先コード', 3: '商品名１', 4: '商品名２', 5: '検収日', 6: '金額（税抜き）', 7: '備考'}
[<Worksheet "請求一覧2101">, <Worksheet "請求一覧2102">, <Worksheet "請求一覧2103">, <Worksheet "請求一覧2105">, <Worksheet "請求一覧2106">, <Worksheet "請求一覧2107">, <Worksheet "請求一覧2108">, <Worksheet "請求一覧2109">, <Worksheet "請求一覧2110">]
[13, 13, 13, 13, 13, 13, 13, 13, 13]


,相手先コード,顧客名
0,1001.0,アデックテクノロジー株式会社
1,1002.0,株式会社サンワテック
2,1003.0,プロト技研株式会社
3,1004.0,株式会社AICUBEシステムズ
4,1005.0,大川情報開発合同会社
5,2001.0,NGCネットワークス株式会社
6,2002.0,光栄情報サービス株式会社
7,3001.0,株式会社クスダ製作所
8,3002.0,大関西リサーチ株式会社
9,4001.0,学校法人 さくら学園


In [54]:
from datetime import datetime
import calendar

seikyu_all = pd.DataFrame()
for i,sheet in enumerate(sheet_list):
    l_2d = get_list_2d(sheet, 4, max_row_list[i], 0, 8)
    name = sheet.title[:4]
    year = "20" + sheet.title[4:6]
    month = sheet.title[6:]
    _, lastday = calendar.monthrange(int(year),int(month))
    date = datetime(int(year),int(month),lastday)
    date_ym = date.strftime('%Y-%m-%d')
    df = pd.DataFrame(l_2d)
    seikyu_df = df.rename(columns=dic_col)
    seikyu_df = seikyu_df.dropna(how='all')
    seikyu_df['年月'] = date_ym
    #seikyu_df = seikyu_df.set_index('年月',drop = True)
    seikyu_all = pd.concat([seikyu_all, seikyu_df])

#display(seikyu_all)

seikyu_sitei = pd.DataFrame()

#seikyu_sitei = seikyu_all[(seikyu_all["相手先コード"] == 1005) &  (seikyu_all["年月"] == "2021-02")]
seikyu_sitei = seikyu_all[seikyu_all["相手先コード"] == 1005]

#請求一覧、顧客一覧データフレームをマージする。
merge_df = pd.merge(seikyu_all,cs_df)

#相手先コード、年月からデータを抽出し、受注No.でソートしたものをデータフレームへ代入   
merge_df = merge_df[(merge_df["相手先コード"] == 1005) &  (merge_df["年月"] == "2021-03-31")].sort_values(by="受注No.",ascending = False)
#年月
date = merge_df["年月"].iloc[0]
#No
no = date[2:4] + date[5:7] + "01"
#会社名
name = merge_df["顧客名"].iloc[0] + "　御中"
print(no,date,name)

#請求元シートへデータを書き込んで、出力する。

#請求元シート読み込み
moto_sheet = wb1["請求書元"]
#row_list = ['A','B','E','F','G','H']
col_list = ['受注No.','商品名１','商品名２','数量','単位','金額（税抜き）','備考']
for i,col in enumerate(col_list):
    try:
        if col == "受注No.":
            for i,data in enumerate(merge_df[col]):
                excel_num = 18 + (i*2)
                cell_num = "A" + str(excel_num)
                print(cell_num,data)
        elif col == "商品名１":
            for i,data in enumerate(merge_df[col]):
                excel_num = 18 + (i*2)
                cell_num = "B" + str(excel_num)
                print(cell_num,data)
        elif col == "商品名２":
            for i,data in enumerate(merge_df[col]):
                excel_num = 18 + (i*2)
                cell_num = "B" + str(excel_num)
                print(cell_num,data)
        elif col == "金額（税抜き）":
            for i,data in enumerate(merge_df[col]):
                excel_num = 18 + (i*2)
                cell_num = "G" + str(excel_num)
                print(cell_num,data)
        elif col == "備考":
            for i,data in enumerate(merge_df[col]):
                excel_num = 18 + (i*2)
                cell_num = "H" + str(excel_num)
                print(cell_num,data)
    except:
        continue

210301 2021-03-31 大川情報開発合同会社　御中
3    210201.0
2    201203.0
Name: 受注No., dtype: float64 A
A18 210201.0
A20 201203.0
3       顧問料
2    社内教育企画
Name: 商品名１, dtype: object B
B18 顧問料
B20 社内教育企画
3    None
2    None
Name: 商品名２, dtype: object B
B18 None
B20 None
3     80000.0
2    150000.0
Name: 金額（税抜き）, dtype: float64 G
G18 80000.0
G20 150000.0
3    2021.2月分
2        None
Name: 備考, dtype: object H
H18 2021.2月分
H20 None
